In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import ks_2samp #for Kolmogorov-Smirnov test

In [ ]:
#models to import
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm 


from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
import time

### Function to calculate cross-validation score

In [ ]:
#getting the cross validation score with different models passed as method
def get_mae(X, y, method):
    return  -1*cross_val_score(method, X, y, scoring = 'neg_mean_squared_log_error', cv = 2)

In [ ]:
import pandas as pd

#import data 
path = ('../input/train.csv')
data = pd.read_csv(path)


In [ ]:
data.head()

In [ ]:
path_2 = ('../input/test.csv')
data_test = pd.read_csv(path_2)
data_test.head()

In [ ]:
#seperate target variables and primary key variable 
data_target = data.target
data_ID = data.ID
data_1 = data.drop(labels = ['target','ID'], axis = 1)

### Analyse the data 

In [ ]:
#DROP cols with constants

zero_stddev_col = []
i=0
for col in data_1.columns:
    #print(data_1[col])
    if int(data_1[col].std()) == float(0): 
        zero_stddev_col.append(col)
    
data_1.drop(zero_stddev_col, axis=1, inplace=True)        

print(len(zero_stddev_col),'cols were removed')
print(zero_stddev_col)

#### Delete coloumns which have too many zeroes

In [ ]:
# to check for non zero values 
np_data = data_1 #copy for checking zeroes

p = np.count_nonzero(np_data, axis = 0)
print('no. of non zero values in each col : ',p)
print('total cols:',len(np_data.columns)) 

col_delete_zer = []
col_keep_zer = []

np_col = np_data.columns

for i in range (0,len(np_col)):
    if np.count_nonzero(np_data.iloc[:,i]) < (data_1.shape[0]/1000):
        col_delete_zer = np.append(col_delete_zer,np_col[i])
    else:
        col_keep_zer = np.append(col_keep_zer,np_col[i])
        

print('no: of cols with too many zeroes',len(col_delete_zer))
print('no: of cols with enough data',len(col_keep_zer))
print()




In [ ]:
#DROP cols with too many zeroes
data_1.drop(col_delete_zer, axis=1, inplace=True)

#### Kolmogorov-Smirnov test to see if the training set and test set have the same distribution. 

If the training set and test set have different distributions then it wont be a good predictor. 


In [ ]:
# Kolmogorov-Smirnov test
THRESHOLD_P_VALUE = 0.01 
THRESHOLD_STATISTIC = 0.2 
diff_cols = []
for col in data_1.columns:
    statistic, pvalue = ks_2samp(data_1[col].values, data_test[col].values)
    if pvalue <= THRESHOLD_P_VALUE and np.abs(statistic) > THRESHOLD_STATISTIC:
        diff_cols.append(col)

print('no of cols not similar in test and train:',len(diff_cols))      


In [ ]:
#DROP cols that fail  Kolmogorov-Smirnov test
print(diff_cols)
data_1.drop(diff_cols, axis=1, inplace=True)


In [ ]:
data_1.shape

In [ ]:
data_test.head()

In [ ]:
test_ID = data_test.ID

In [ ]:
#data_test modification for model training
t= time.time()
data_test.drop(labels = ['ID'], axis=1, inplace=True)
data_test.drop(zero_stddev_col, axis=1, inplace=True)
data_test.drop(col_delete_zer, axis=1, inplace=True)
data_test.drop(diff_cols, axis=1, inplace=True)

print('IDs',test_ID.shape)
print('test data', data_test.shape)
print(t-time.time())

#### Graphical analysis

In [ ]:
log_target = np.log(data_target)
plt.hist(log_target, bins=50)
plt.title('Histogram : Target')
plt.xlabel('Bins')
plt.ylabel('Counts')
plt.show()

#COnclusion: definately aggregation towards rounded values



In [ ]:
#train, test = train_test_split(data, train_size=0.55, test_size=0.45, random_state=0)

#train_target = train.target
#train = train.drop(labels = ['ID','target'], axis = 1)

#test_target = test.target
#test = test.drop(labels = ['ID','target'], axis = 1)

In [ ]:
#XGB_tt_split = XGBRegressor(early_stopping_rounds=5)
#XGB_tt_split.fit(train, train_target)
#XGB_pred = XGB_tt_split.predict(test)

#XGB_tt_split_score = mean_squared_log_error(test_target, XGB_pred)

#print('XGB',XGB_tt_split_score)

In [ ]:
#Prediction based on LightGBM

#t_LGBM = time.time()
#LGB_1 = lightgbm.LGBMRegressor(objective='regression',metric='rmse',is_training_metric=True)
#LGB_1.fit(data_1, log_target)
#LGB_pred = LGB_1.predict(data_test)
#print(time.time()-t_LGBM)

#LGB_tt_split_score = mean_squared_log_error(test_target, LGB_pred)

#print('LGB',LGB_tt_split_score)

In [ ]:
#Plotting of LightGBM prediction 
#t_LGBM_plt = time.time()
#plt.hist(LGB_pred, bins=100)
#plt.title('Histogram : prediction')
#plt.xlabel('Bins')
#plt.ylabel('Counts')
#plt.show()
#print(time.time()-t_LGBM_plt)

In [ ]:
#LGB_pred = np.exp(LGB_pred)

In [ ]:
#my_submission_LGBM = pd.DataFrame({'ID': test_ID, 'target': LGB_pred})
#my_submission_LGBM.to_csv('submission_LGBM.csv', index=False)

In [ ]:
t_RF = time.time()
RF_1 = RandomForestRegressor()
RF_1.fit(data_1, data_target)
RF_pred = RF_1.predict(data_test)
print(time.time()-t_RF)

In [ ]:
#Plotting of RF prediction 

#t_RF_plt = time.time()
#plt.hist(RF_pred, bins=100)
#plt.title('Histogram : prediction')
#plt.xlabel('Bins')
#plt.ylabel('Counts')
#plt.show()
#print(time.time()-t_RF_plt)

In [ ]:
t_LGB = time.time()
LGB_1 = lightgbm.LGBMRegressor(objective='regression',metric='rmse',is_training_metric=True)
LGB_1.fit(data_1, data_target)
LGB_pred = LGB_1.predict(data_test)
print(time.time()-t_LGB)

### creating an average prediction from two different models

In [ ]:
sum_pred = (LGB_pred+RF_pred)/2

In [ ]:
my_submission_ensemble = pd.DataFrame({'ID': test_ID, 'target': sum_pred})
my_submission_ensemble.to_csv('submission_ensemble.csv', index=False)